<a href="https://colab.research.google.com/github/muratal49/LDA-Topic-Selection-News-Articles/blob/main/LDA_Topic_Modelling_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will be working on news articles and classify them buy their topics which can more than one depending on the content.  
**LDA models** start with the approach that both documents have a distribution of different topics, and each word in the corpus have different  distribution for each specific topic.

In this project, we will work on the scikit-learns News data set that has 3899 articles from 20 different Newsgroups.

In [3]:
#First pulling in the documents:

from sklearn.datasets import fetch_20newsgroups

# Load dataset (selecting a few categories)
categories = ['rec.sport.hockey', 'comp.graphics', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert to list of documents
documents = newsgroups.data
print(f"Loaded {len(documents)} news articles.")

Loaded 3899 news articles.


In [10]:
print(documents[0])




Hey tough guy, freedom necessitates responsibility, and
no freedom is absolute.  
BTW, to anyone who defends Arafat, read on:

"Open fire on the new Jewish immigrants, be they from the Soviet
Union, Ethiopia or anywhere else....I give you my instructions to
use violence against the immigrants.  I willjail anyone who
refuses to do this."
				Yassir Arafat, Al-Muharar, 4/10/90

At least he's not racist!
Just anti-Jewish



In [8]:
# Required libraries and tools:
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

In [19]:
#lets do the first processing:
#Tokenization, Stopwors removal and lammatization:

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Tokenization and cleaning
def preprocess(text):
    return [word for word in simple_preprocess(text) if word not in stop_words]

# Apply preprocessing

processed_docs = [preprocess(doc) for doc in documents]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [23]:
processed_docs[0][:10]

['hey',
 'tough',
 'guy',
 'freedom',
 'necessitates',
 'responsibility',
 'freedom',
 'absolute',
 'btw',
 'anyone']

the variblae

**corpus:** creates a list of size as processed_docs and each element in the list is BOW for that doc

In [25]:
#Bag of words for our corpus:
from gensim.corpora import Dictionary


dictionary = Dictionary(processed_docs)

corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [31]:
once_tokens = [dictionary[token_id] for token_id, freq in dictionary.dfs.items() if freq == 1]


In [32]:
once_tokens[:10]

['necessitates',
 'willjail',
 'liverpool',
 'vanecek',
 'tres',
 'spif',
 'newsfeed',
 'spalling',
 'lasts',
 'inconvienenced']

# It is time to train our **LDA** Model:

In [60]:
from gensim.models import LdaModel

# Train the LDA model
# Num of topics is used entry: We select 4, LDA will cluster docs into 4, you can also name
#them later when you see what those clusters are about.
lda_model = LdaModel(corpus=corpus, num_topics=4, id2word=dictionary, passes=10)

# Print the top words for each topic
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.007*"space" + 0.005*"people" + 0.005*"israel" + 0.004*"jews" + 0.003*"armenian"')
(1, '0.007*"pit" + 0.006*"la" + 0.006*"det" + 0.006*"gm" + 0.006*"period"')
(2, '0.008*"game" + 0.007*"said" + 0.007*"people" + 0.005*"like" + 0.005*"know"')
(3, '0.011*"image" + 0.008*"edu" + 0.008*"graphics" + 0.008*"jpeg" + 0.006*"file"')


So from above we can see that
*   0: Politics
*   1: Science
*   2: Computers
*   3: Sports



In [47]:
new_doc = "NASA is planning a mission to Mars next year using the advanced computer graphics date processing ."
bow_new_doc = dictionary.doc2bow(preprocess(new_doc))

# Get topic distribution
topic_distribution = lda_model[bow_new_doc]
print(topic_distribution)

[(0, 0.019474424), (1, 0.68809474), (2, 0.27263343), (3, 0.01979741)]


In [50]:
pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.3 MB/s eta 0:00:00


In [51]:
import pyLDAvis.gensim

# Prepare visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.059176  0.175465       1        1  33.887077
1      0.041457  0.042542       2        1  28.433147
2      0.210025 -0.082778       3        1  21.853653
3     -0.192305 -0.135229       4        1  15.826123, topic_info=          Term         Freq        Total Category  logprob  loglift
158      image  1150.000000  1150.000000  Default  30.0000  30.0000
526      space  1367.000000  1367.000000  Default  29.0000  29.0000
1619  graphics   803.000000   803.000000  Default  28.0000  28.0000
699       game   750.000000   750.000000  Default  27.0000  27.0000
5736      jpeg   774.000000   774.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
938       last   206.505541   643.127151   Topic4  -5.9052   0.7075
21         new   215.247822   903.472821   Topic4  -5.8637   0.4091
261        one   195.678845  2137.761674   Topic4  -5.9591  -0.5475
437      april   163.359414   340.012402   Topic4  -6.1396   1.1105
751        two   171.141924   931.722701   Topic4  -6.0930   0.1490

[310 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
17651      4  0.998086          adl
11097      2  0.987657  advertising
65         1  0.282626         also
65         2  0.323522         also
65         3  0.319140         also
...      ...       ...          ...
543        1  0.458361        years
543        2  0.413149        years
543        3  0.020268        years
543        4  0.107575        years
8202       3  0.996731          zip

[586 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4])

**Let's improve our model using TF-IDF before LDA:**

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert processed documents back to text
text_documents = [" ".join(doc) for doc in processed_docs]

# Apply TF-IDF
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(text_documents)

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
feature_names[:10]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['aa', 'aao', 'aaoepp', 'aaplay', 'aarnet', 'aas', 'aawin', 'ab',
       'ababa', 'abandon'], dtype=object)

In [55]:
#Keeping the high Tf_IDF words, applying the word filtering based on featere names extracted above:

important_words = set(feature_names)

# Filter processed docs to keep only important words
filtered_docs = [[word for word in doc if word in important_words] for doc in processed_docs]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:

#Now using the same steps in LDA, this time using the filtered_docs dictionary with filtered words
dictionary_TDIDF = Dictionary(filtered_docs)

# Convert documents into bag-of-words format
corpus_TDIDF = [dictionary.doc2bow(doc) for doc in filtered_docs]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
# Train LDA model
lda_model_with_TDIDF = LdaModel(corpus=corpus_TDIDF, num_topics=4, id2word=dictionary_TDIDF, passes=10)

# Print topics
topics_TDIDF = lda_model_with_TDIDF.print_topics(num_words=5)
for topic in topics_TDIDF:
    print(topic)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.010*"edu" + 0.009*"graphics" + 0.007*"data" + 0.005*"image" + 0.005*"available"')
(1, '0.015*"jpeg" + 0.014*"image" + 0.009*"file" + 0.008*"gif" + 0.006*"bit"')
(2, '0.008*"people" + 0.007*"armenian" + 0.006*"armenians" + 0.006*"said" + 0.005*"israel"')
(3, '0.008*"space" + 0.006*"game" + 0.005*"like" + 0.005*"team" + 0.004*"think"')


In [61]:
# Prepare visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.049654  0.149945       1        1  41.911296
3     -0.233415 -0.086830       2        1  25.507229
2      0.127021  0.102429       3        1  22.745708
1      0.156048 -0.165543       4        1   9.835766, topic_info=          Term         Freq        Total Category  logprob  loglift
129      image  1194.000000  1194.000000  Default  30.0000  30.0000
1443  graphics   834.000000   834.000000  Default  29.0000  29.0000
5122      jpeg   804.000000   804.000000  Default  28.0000  28.0000
604       game   775.000000   775.000000  Default  27.0000  27.0000
36         edu  1088.000000  1088.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1012      play   120.517781   410.690625   Topic4  -5.8208   1.0931
14         new   138.071045   929.987433   Topic4  -5.6848   0.4117
36         edu   131.343641  1088.862873   Topic4  -5.7348   0.2041
524      think   126.905397   940.584296   Topic4  -5.7692   0.3161
839       year   116.459506   696.123088   Topic4  -5.8551   0.5312

[294 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
3556       2  0.013384         abc
3556       3  0.963635         abc
3556       4  0.026768         abc
12029      4  0.987028  adirondack
13819      1  0.997554         adl
...      ...       ...         ...
466        1  0.635292       years
466        2  0.032070       years
466        3  0.306956       years
466        4  0.024434       years
7190       2  0.992717         zip

[533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])